In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import numpy as np

# = = = = = = = = = = = = = = =

import re
import html as html_lib

In [2]:
directory_in = './file/1.part_number'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'part_number.xlsx'

In [3]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：10



,No,Manufacturer,Part Number
0,1,FOUR SEASONS,33072
1,2,FOUR SEASONS,33188
2,3,FOUR SEASONS,56790
3,4,FOUR SEASONS,33094
4,5,FOUR SEASONS,33155
5,6,FOUR SEASONS,55307
6,7,FOUR SEASONS,33098
7,8,FOUR SEASONS,83107
8,9,FOUR SEASONS,83044
9,10,FOUR SEASONS,83052


In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                         1
 Manufacturer    FOUR SEASONS
 Part Number            33072
 Name: 0, dtype: object)

In [5]:
import time

data = {'dopartsearch': '1',
        'partsearch[partnum][partsearch_007]': crawler_series['Part Number'],
        'partsearch[manufacturer][partsearch_007]': crawler_series['Manufacturer'],
        'partsearch[do][partsearch_007]': '1',
        'func': 'sendparttabsearch',
        'payload': '{}',
        'api_json_request': '1'}

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_check = f'''https://www.rockauto.com/en/partsearch/?mfr={crawler_series['Manufacturer']}&partnum={crawler_series['Part Number']}'''

        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                             data=data,
                             headers=get_header(),
                             proxies=get_proxy(),
                             timeout=(5, 15))

        if resp.status_code == 200:
            dict_ = resp.json()

            if 'redirect_to_url' not in dict_ and 'collected_javascript' in dict_:
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

dict_

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


{'searchnoderesults': '<div class="ranavnode" id="nav[__GIP__1__]"><input type="hidden" autocomplete="off" id="jsn[__GIP__1__]" value="{&quot;groupindex&quot;:&quot;__GIP__1__&quot;,&quot;tab&quot;:&quot;partsearch&quot;,&quot;idepth&quot;:0,&quot;parttype&quot;:&quot;6972&quot;,&quot;catalogname&quot;:&quot;FOUR SEASONS&quot;,&quot;partnumber&quot;:&quot;33072&quot;,&quot;nodetype&quot;:&quot;parttabsearch&quot;,&quot;expanded&quot;:true,&quot;loaded&quot;:true}" /><input type="hidden" autocomplete="off" name="navnodeunique[partsearch-6972]" id="navnodeunique[partsearch-6972]" value="__GIP__1__" /><div id="nav_o[__GIP__1__]" class="ranavouter"><div class="inner"><table class="tbl"  cellpadding="0" cellspacing="0"><tr><td class="niconspace nexpandedicon" id="navicon[__GIP__1__]"><a href="/en/parts/four+seasons,33072,a/c+receiver+drier+/+accumulator,6972" onclick="cataloglite.LinkIntercept_ToggleNavNode(&quot;__GIP__1__&quot;); return false;" tabindex="-1"><img src="/Images/1pxtranspare

In [6]:
soup = BeautifulSoup(dict_['searchnoderesults'], 'lxml')
html = etree.HTML(str(soup))

with open('./part.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div class="ranavnode" id="nav[__GIP__1__]">
   <input autocomplete="off" id="jsn[__GIP__1__]" type="hidden" value='{"groupindex":"__GIP__1__","tab":"partsearch","idepth":0,"parttype":"6972","catalogname":"FOUR SEASONS","partnumber":"33072","nodetype":"parttabsearch","expanded":true,"loaded":true}'/>
   <input autocomplete="off" id="navnodeunique[partsearch-6972]" name="navnodeunique[partsearch-6972]" type="hidden" value="__GIP__1__"/>
   <div class="ranavouter" id="nav_o[__GIP__1__]">
    <div class="inner">
     <table cellpadding="0" cellspacing="0" class="tbl">
      <tr>
       <td class="niconspace nexpandedicon" id="navicon[__GIP__1__]">
        <a href="/en/parts/four+seasons,33072,a/c+receiver+drier+/+accumulator,6972" onclick='cataloglite.LinkIntercept_ToggleNavNode("__GIP__1__"); return false;' tabindex="-1">
         <img alt="Toggle" src="/Images/1pxtransparent.png"/>
        </a>
       </td>
       <td class="nlabel">
        <a class="navlabellink nvoff

In [7]:
list_tbody = html.xpath('//tbody[contains(@id, "listingcontainer[__GIP__")]')

list_tbody

[<Element tbody at 0x11b525280>]

In [8]:
list_dict = [json.loads(tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/input[contains(@id, "listing_data_essential[__GIP__")]/@value')[0]) for tbody in list_tbody]

list_dict

[{'groupindex': '__GIP__3__',
  'carcode': 0,
  'parttype': '6972',
  'partkey': '814749',
  'opts': {'0-0-8-1': {'warehouse': '77479',
    'whpartnum': 'FOR 33072:8',
    'optionlist': '0',
    'paramcode': '0',
    'notekey': '8',
    'multiple': '1'},
   '0-0-1-1': {'warehouse': '77481',
    'whpartnum': 'FOR 33072',
    'optionlist': '0',
    'paramcode': '0',
    'notekey': '1',
    'multiple': '1'},
   '0-0-0-1': {'warehouse': '23220',
    'whpartnum': 'FOR 33072',
    'optionlist': '0',
    'paramcode': '0',
    'notekey': '0',
    'multiple': '1'}}}]

In [9]:
list_type_code, list_part_code = [dict_['parttype'] for dict_ in list_dict], [dict_['partkey'] for dict_ in list_dict]

list_type_code, list_part_code

(['6972'], ['814749'])

In [10]:
list_dict_2 = [json.loads(tbody.xpath('./tr[1]/td/input[contains(@id, "listing_data_supplemental[__GIP__")]/@value')[0]) for tbody in list_tbody]

list_dict_2

[{'partnumber': '33072',
  'catalogname': 'FOUR SEASONS',
  'belongstolisting': '__GIP__2__',
  'sortgroup': 0,
  'sortgrouptext': '',
  'paramdesc': ''}]

In [11]:
list_part_number, list_manufacturer, list_choose = [dict_['partnumber'] for dict_ in list_dict_2], [dict_['catalogname'] for dict_ in list_dict_2], [dict_['paramdesc'].strip() for dict_ in list_dict_2]

list_part_number, list_manufacturer, list_choose

(['33072'], ['FOUR SEASONS'], [''])

In [12]:
array_index_part_number, array_index_manufacturer = np.where(np.array(list_part_number) == crawler_series['Part Number'])[0], np.where(np.array(list_manufacturer) == crawler_series['Manufacturer'])[0]

array_index_part_number, array_index_manufacturer

(array([0]), array([0]))

In [13]:
array_index_part = np.intersect1d(array_index_part_number, array_index_manufacturer)

array_index_part

array([0])

In [14]:
if array_index_part.size == 1:
    index_part = array_index_part[0]

index_part

np.int64(0)

In [15]:
url_check

'https://www.rockauto.com/en/partsearch/?mfr=FOUR SEASONS&partnum=33072'

In [16]:
list_span = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row-moreinfo-truck"]/span[@class="span-link-underline-remover"]')
list_note = [etree.tostring(span).decode('utf-8') for span in list_span]
list_note = [re.sub(r'<.*?>', '', note) for note in list_note]
list_note = [re.sub(r' +', ' ', note.strip()) for note in list_note]
list_note = [html_lib.unescape(note) for note in list_note]

list_note

[]

In [17]:
dict_note, i = {}, 0
for note in list_note:
    if note != '':
        dict_note[str(i)] = note

dict_note

{}

In [18]:
list_a = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]')
info = list_a[0].xpath('./@href')[0].strip() if list_a else ''
info = info if info.startswith('https://www.rockauto.com/en/moreinfo.php?') else f'https://www.rockauto.com/en/moreinfo.php?pk={list_part_code[index_part]}&pt={list_type_code[index_part]}&Lennon=1'

info

'https://www.rockauto.com/en/moreinfo.php?pk=814749&cc=0&pt=6972'

In [19]:
list_div = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row"]/span[contains(@title, "Please make sure")]/parent::div[@class="listing-text-row"]')
list_note = [etree.tostring(div).decode('utf-8') for div in list_div]
list_note = [re.sub(r'<.*?>', '', note) for note in list_note]
list_note = [re.sub(r' +', ' ', note.strip()) for note in list_note]
list_note = [html_lib.unescape(note) for note in list_note]

list_note

['Category: A/C Receiver Drier / Accumulator']

In [20]:
dict_note_2, i = {}, 0
for note in list_note:
    if note != '':
        dict_note_2[str(i)] = note

dict_note_2

{'0': 'Category: A/C Receiver Drier / Accumulator'}

In [21]:
list_list_text = [text.split(',') for td in list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]') for text in td.xpath('./span[contains(@title, "Replaces these Alternate")]/text()')]
list_text = [text.strip() for list_text in list_list_text for text in list_text if text.strip() != '']
oem = ';'.join(list(dict.fromkeys(list_text)))

oem

'151911;19188493;88919275;88919343;F49Z19C836B'

In [22]:
list_input = list_tbody[index_part].xpath('./tr[1]/td/div[contains(@class, "listing-image-space-occupy")]/input[contains(@id, "jsninlineimg[__GIP__")]')
list_dict = [json.loads(value) for input_ in list_input for value in input_.xpath('./@value')]

list_dict

[{'Size': 'Thumb',
  'SlotIndex': 0,
  'Slots': [{'TitlePhrase': None,
    'ParamCodes': [None],
    'Type': 'Single',
    'ImageData': {'Thumb': '/info/52/33072__ra_m.jpg',
     'Popup': '/info/52/33072.jpg',
     'Full': '/info/52/33072.jpg'}},
   {'TitlePhrase': None,
    'ParamCodes': [None],
    'Type': 'Single',
    'ImageData': {'Thumb': '/info/52/33072FRO__ra_m.jpg',
     'Popup': '/info/52/33072FRO.jpg',
     'Full': '/info/52/33072FRO.jpg'}},
   {'TitlePhrase': None,
    'ParamCodes': [None],
    'Type': 'Single',
    'ImageData': {'Thumb': '/info/52/33072RSID__ra_m.jpg',
     'Popup': '/info/52/33072RSID.jpg',
     'Full': '/info/52/33072RSID.jpg'}},
   {'TitlePhrase': None,
    'ParamCodes': [None],
    'Type': 'Single',
    'ImageData': {'Thumb': '/info/52/33072LSID__ra_m.jpg',
     'Popup': '/info/52/33072LSID.jpg',
     'Full': '/info/52/33072LSID.jpg'}},
   {'TitlePhrase': None,
    'ParamCodes': [None],
    'Type': 'Single',
    'ImageData': {'Thumb': '/info/52/33072BA

In [23]:
dict_src, i = {}, 0
for dict_ in list_dict:
    list_src = dict_.get('Slots', [])
    for src in list_src:
        dict_src[str(i)] = f'''https://www.rockauto.com{src['ImageData']['Full'].strip()}'''
        i += 1

dict_src

{'0': 'https://www.rockauto.com/info/52/33072.jpg',
 '1': 'https://www.rockauto.com/info/52/33072FRO.jpg',
 '2': 'https://www.rockauto.com/info/52/33072RSID.jpg',
 '3': 'https://www.rockauto.com/info/52/33072LSID.jpg',
 '4': 'https://www.rockauto.com/info/52/33072BAC.jpg',
 '5': 'https://www.rockauto.com/info/52/33072TOP.jpg'}

In [24]:
list_span = list_tbody[index_part].xpath('./tr[1]/td[contains(@id, "listingtd[__GIP__") and contains(@id, "__][price]")]/span[contains(@id, "dprice[__GIP__") and contains(@id, "__][td]")]/span/span[contains(@id, "dprice[__GIP__") and contains(@id, "__][v]")]')
list_price = [etree.tostring(span).decode('utf-8') for span in list_span]
list_price = [re.sub(r'<.*?>', '', price) for price in list_price]
list_price = [re.sub(r' +', ' ', price.strip()) for price in list_price]
list_price = [html_lib.unescape(price) for price in list_price]
dict_price = {'0': '; '.join(list_price)}

dict_price

{'0': '$5.76'}

In [25]:
list_td = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]')
list_price = [etree.tostring(option).decode('utf-8') for td in list_td for option in td.xpath('./div/div[contains(@class, "listing-optionchoice-multiple-container")]/span[contains(@id, "ddrepl[optionchoice[__GIP__") and contains(@id, "__]][container]")]/span[@class="hide-if-js"]/select[contains(@id, "optionchoice[__GIP__") and contains(@id, "__]")]/option[@value!=""]')]
list_price = [re.sub(r'<.*?>', '', price) for price in list_price]
list_price = [re.sub(r' +', ' ', price.strip()) for price in list_price]
list_price = [html_lib.unescape(price) for price in list_price]
if list_price:
    dict_price['1'] = {}
    for i, price in enumerate(list_price):
        dict_price['1'][str(i)] = price

dict_price

{'0': '$5.76',
 '1': {'0': '[Wholesaler Closeout - Private Label Pkg. - 30 Day Warranty] ($5.76) ^',
  '1': '[Wholesaler Closeout -- 30 Day Warranty] ($14.18) ^',
  '2': '[Regular Inventory] ($22.79) ^'}}

In [26]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Manufacturer': crawler_series['Manufacturer'],
                          'Part Number': crawler_series['Part Number'],
                          'OEM': oem,
                          'Picture': '',
                          'Url': f'''https://www.rockauto.com/en/parts/{crawler_series['Manufacturer']},{crawler_series['Part Number']},Lennon,{list_type_code[index_part]}''',
                          'Json_Src': json.dumps(dict_src),
                          'Info': info,
                          'Json_Note 1': json.dumps(dict_note),
                          'Json_Note 2': json.dumps(dict_note_2),
                          'Choose': list_choose[index_part],
                          'Json_Price': json.dumps(dict_price),
                          'Type Code': list_type_code[index_part],
                          'Part Code': list_part_code[index_part],
                          'JOIN_MPNTCPC': f'''{crawler_series['Manufacturer']};{crawler_series['Part Number']};{list_type_code[index_part]};{list_part_code[index_part]}'''})

list_dict_correct

[{'No': 1,
  'Manufacturer': 'FOUR SEASONS',
  'Part Number': '33072',
  'OEM': '151911;19188493;88919275;88919343;F49Z19C836B',
  'Picture': '',
  'Url': 'https://www.rockauto.com/en/parts/FOUR SEASONS,33072,Lennon,6972',
  'Json_Src': '{"0": "https://www.rockauto.com/info/52/33072.jpg", "1": "https://www.rockauto.com/info/52/33072FRO.jpg", "2": "https://www.rockauto.com/info/52/33072RSID.jpg", "3": "https://www.rockauto.com/info/52/33072LSID.jpg", "4": "https://www.rockauto.com/info/52/33072BAC.jpg", "5": "https://www.rockauto.com/info/52/33072TOP.jpg"}',
  'Info': 'https://www.rockauto.com/en/moreinfo.php?pk=814749&cc=0&pt=6972',
  'Json_Note 1': '{}',
  'Json_Note 2': '{"0": "Category: A/C Receiver Drier / Accumulator"}',
  'Choose': '',
  'Json_Price': '{"0": "$5.76", "1": {"0": "[Wholesaler Closeout - Private Label Pkg. - 30 Day Warranty] ($5.76) ^", "1": "[Wholesaler Closeout -- 30 Day Warranty] ($14.18) ^", "2": "[Regular Inventory] ($22.79) ^"}}',
  'Type Code': '6972',
  'Par

In [27]:
crawler_status = True

crawler_status

True

In [28]:
crawler_series['Check_Url'] = url_check

list_series_error.append(crawler_series)

list_series_error

[No                                                              1
 Manufacturer                                         FOUR SEASONS
 Part Number                                                 33072
 Check_Url       https://www.rockauto.com/en/partsearch/?mfr=FO...
 Name: 0, dtype: object]

In [29]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_part.xlsx', index=False)

df_correct

,No,Manufacturer,Part Number,OEM,Picture,Url,Json_Src,Info,Json_Note 1,Json_Note 2,Choose,Json_Price,Type Code,Part Code,JOIN_MPNTCPC
0,1,FOUR SEASONS,33072,151911;19188493;88919275;88919343;F49Z19C836B,,https://www.rockauto.com/en/parts/FOUR SEASONS...,"{""0"": ""https://www.rockauto.com/info/52/33072....",https://www.rockauto.com/en/moreinfo.php?pk=81...,{},"{""0"": ""Category: A/C Receiver Drier / Accumula...",,"{""0"": ""$5.76"", ""1"": {""0"": ""[Wholesaler Closeou...",6972,814749,FOUR SEASONS;33072;6972;814749


In [30]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_part_error.xlsx', index=False)

df_error

,No,Manufacturer,Part Number,Check_Url
0,1,FOUR SEASONS,33072,https://www.rockauto.com/en/partsearch/?mfr=FO...
